In [ ]:
import numpy as np
import os
import pandas as pd
import zipfile


file_path = '/homework_4.zip'

# Вилучення архіву
with zipfile.ZipFile(file_path, 'r') as zip_ref:
    zip_ref.extractall('/extracted_folder')
extracted_folder_path = '/extracted_folder'

# Список для зберігання даних
data_list = []

# Цикл по всім папкам (idle, running, stairs, walking)
for folder_name in os.listdir(os.path.join(extracted_folder_path, 'data')):
    folder_path = os.path.join(extracted_folder_path, 'data', folder_name)
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.csv'):
            file_path = os.path.join(folder_path, file_name)
            df = pd.read_csv(file_path)
            df['action'] = folder_name  # Додавання стовпця під назвою папки
            data_list.append(df)

# Об'єднання даних зі списку до однієї таблиці
data_df = pd.concat(data_list, ignore_index=True)
data_df.head()

Mounted at /content/drive


,accelerometer_X,accelerometer_Y,accelerometer_Z,action
0,6.090849,6.928820,-8.298302,running
1,22.424093,24.665066,15.121778,running
2,-3.615245,7.704541,-6.751649,running
3,26.647465,27.461494,8.336611,running
4,2.336741,2.394202,0.488417,running


In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


X = data_df.drop('action', axis=1)
y = data_df['action']

# Поділ даних на навчальну та тестову вибірки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Розрахунок часових ознак
X_train['time_mean'] = X_train.mean(axis=1)
X_test['time_mean'] = X_test.mean(axis=1)

# Нормалізація даних
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Навчання моделей
svm_model = SVC()
rf_model = RandomForestClassifier()
svm_model.fit(X_train_scaled, y_train)
rf_model.fit(X_train_scaled, y_train)

# Передбачення
svm_pred = svm_model.predict(X_test_scaled)
rf_pred = rf_model.predict(X_test_scaled)

# Оцінка моделей
svm_accuracy = accuracy_score(y_test, svm_pred)
rf_accuracy = accuracy_score(y_test, rf_pred)

svm_f1 = f1_score(y_test, svm_pred, average='weighted')
rf_f1 = f1_score(y_test, rf_pred, average='weighted')
print("Accuracy (SVM):", svm_accuracy)
print("Accuracy (Random Forest):", rf_accuracy)
print("F1 Score (SVM):", svm_f1)
print("F1 Score (Random Forest):", rf_f1)

svm_report = classification_report(y_test, svm_pred)
print("SVM Results:")
print(svm_report)

rf_report = classification_report(y_test, rf_pred)
print("Random Forest Results:")
print(rf_report)

Accuracy (SVM): 0.8953368410192922
Accuracy (Random Forest): 0.9994841638295677
F1 Score (SVM): 0.8851157144829856
F1 Score (Random Forest): 0.9994837692942329
SVM Results:
              precision    recall  f1-score   support

        idle       0.96      0.99      0.98      6254
     running       0.93      0.90      0.92     20326
      stairs       1.00      0.00      0.01       964
     walking       0.81      0.90      0.85     11228

    accuracy                           0.90     38772
   macro avg       0.92      0.70      0.69     38772
weighted avg       0.90      0.90      0.89     38772

Random Forest Results:
              precision    recall  f1-score   support

        idle       1.00      1.00      1.00      6254
     running       1.00      1.00      1.00     20326
      stairs       1.00      0.99      1.00       964
     walking       1.00      1.00      1.00     11228

    accuracy                           1.00     38772
   macro avg       1.00      1.00      1.00